# 03 - Entrenamiento y Evaluación de Modelos

**Objetivo:** En este notebook, se entrenarán diferentes modelos de clasificación utilizando los datos preprocesados. Se evaluará su rendimiento con métricas apropiadas para datasets desbalanceados y se compararán para seleccionar el mejor.

## 1. Carga de Datos y Librerías
Cargamos los conjuntos de datos de entrenamiento y prueba que fueron preparados y guardados en el notebook anterior.

In [3]:
import pandas as pd
from joblib import load
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Carga los datos preprocesados desde el archivo joblib
datos = load('../data/processed/cicids2017_preprocessed_data.joblib')

X_train_scaled = datos['X_train_scaled']
X_test_scaled = datos['X_test_scaled']
y_train = datos['y_train']
y_test = datos['y_test']

print("Datos cargados. Formas:")
print("X_train_scaled:", X_train_scaled.shape)
print("X_test_scaled:", X_test_scaled.shape)

Datos cargados. Formas:
X_train_scaled: (1979513, 39)
X_test_scaled: (848363, 39)


## 2. Modelo Base: Regresión Logística

Comenzamos con un modelo simple y rápido como la Regresión Logística para establecer un rendimiento de referencia (baseline). Cualquier modelo más complejo que probemos después deberá superar a este.

In [4]:
# Inicializamos el modelo de Regresión Logística
# n_jobs = -1 usa todos los núcleos de tu CPU para acelerar el entrenamiento
lr_model = LogisticRegression(random_state = 42, n_jobs = -1, max_iter = 1000)

# Entrenamos el modelo SÓLO con los datos de entrenamiento
print("Entrenando el modelo de Regresión Logística...")
lr_model.fit(X_train_scaled, y_train)

Entrenando el modelo de Regresión Logística...


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


### 2.1. Evaluación del Modelo Base

Generamos el reporte de clasificación para analizar el rendimiento del modelo en el conjunto de prueba. Nos enfocamos en **Precision**, **Recall** y **F1-Score** debido al desbalance de clases.

In [5]:
# Hacemos predicciones sobre el conjunto de prueba
y_pred = lr_model.predict(X_test_scaled)

# Generamos y mostramos el reporte de clasificación
print("\nReporte de Clasificación - Regresión Logística:")
print(classification_report(y_test, y_pred))


Reporte de Clasificación - Regresión Logística:


c:\Users\brand\anaconda3\envs\cyber-project-py311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\brand\anaconda3\envs\cyber-project-py311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


                            precision    recall  f1-score   support

                    BENIGN       0.98      0.98      0.98    681396
                       Bot       0.93      0.02      0.05       587
                      DDoS       0.91      0.99      0.95     38408
             DoS GoldenEye       0.93      0.74      0.82      3088
                  DoS Hulk       0.98      0.87      0.92     69037
          DoS Slowhttptest       0.87      0.77      0.81      1650
             DoS slowloris       0.89      0.47      0.61      1739
               FTP-Patator       0.96      0.33      0.49      2380
                Heartbleed       0.43      1.00      0.60         3
              Infiltration       0.14      0.09      0.11        11
                  PortScan       0.81      1.00      0.89     47641
               SSH-Patator       0.94      0.51      0.66      1769
  Web Attack � Brute Force       0.00      0.00      0.00       452
Web Attack � Sql Injection       0.00      0.00

c:\Users\brand\anaconda3\envs\cyber-project-py311\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


### Análisis de Resultados (Regresión Logística)

El reporte de clasificación de nuestro modelo base de Regresión Logística nos da varias pistas importantes sobre su rendimiento en los datos desbalanceados.

**Fortalezas** 
- El modelo es extremadamente bueno para identificar el tráfico BENIGN, con un f1-score de 0.98.

- También muestra un rendimiento muy alto para las clases de ataques más comunes en el dataset, como DDoS (0.95), DoS Hulk (0.92) y PortScan (0.89). Particularmente, el recall para PortScan y DDoS es casi perfecto (1.00 y 0.99), lo que significa que el modelo es muy eficaz para no dejar pasar estos tipos de ataques.

**Debilidades**
- El modelo es completamente incapaz de identificar cualquiera de los ataques de tipo Web Attack (Brute Force, Sql Injection, XSS). Obtuvo un 0.00 en todas las métricas para estas clases, lo que significa que nunca predijo una de ellas. Esto es la causa de la advertencia UndefinedMetricWarning.

- El rendimiento es muy pobre para otras clases minoritarias. Por ejemplo, Bot tiene un recall de solo 0.02 (detecta solo el 2% de los ataques de Bot) y FTP-Patator un recall de 0.33 (deja pasar 2 de cada 3 ataques).

**Conclusión General**

El modelo base está fuertemente sesgado hacia las clases con más datos. Es muy competente para la tarea de "Benigno vs. Ataque Común", pero ignora casi por completo los ataques más raros y con menos ejemplos (support).

El objetivo principal para los siguientes modelos será mejorar drásticamente el recall de estas clases minoritarias, ya sea con un algoritmo más complejo o con técnicas de balanceo de datos.